# **Assignment 2 - MS Spectra**

# Initialization

Connecting to Google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Checking the working adress.

In [ ]:
!ls

/content/drive/MyDrive/DSU
synthetic_search_results_updated_bins.xml  synthetic_search_results.xml
synthetic_search_results_updated.xml


Importing libraries.

In [ ]:
# Directory checker
from os import getcwd, listdir
from os.path import isfile

# XML interaction
import lxml
from lxml import etree as et
import xml.etree.ElementTree as ET

# HTML printout
from IPython.display import display, HTML
# XML printout
from bs4 import BeautifulSoup

# mathematical functions
from collections import Counter


# graphical output
import matplotlib.pyplot as plt

# **Tasks**

# Extract

## I. Read XML file into the selected workspace.

Add data into Files and check if present in current working directory.

In [ ]:
# Path to the synthetic_search_results.xml file
# xml_file_path = "/content/drive/MyDrive/Colab Notebooks/project/synthetic_search_results.xml"
# xml_file_path = "/content/drive/MyDrive/DSU/synthetic_search_results.xml"
xml_file_path = "synthetic_search_results.xml"

if not isfile(xml_file_path):
  raise RuntimeError(f"Please load synthetic_search_results.xml file before proceeding.\n"
                     f"load the file in current dir: '{getcwd()}' "
                     f"with following content:\n{listdir()}")

Function to compile xslt language in python and print HTML response.

In [ ]:
def transform_xml(xml, xsl, print=True, file=None):
  if isfile(xsl):
    xslt_doc = et.parse(xsl)
  else:
    xslt_doc = et.XML(xsl)

  xslt_transformer = et.XSLT(xslt_doc)

  source_doc = et.parse(xml)
  output_doc = xslt_transformer(source_doc)
  return output_doc

def publish_html(output_doc, file=None):
  if file:
    if file.endswith(".html"):
      output_doc.write(file)
  else:
    display(HTML(str(output_doc)))


def publish_xml(output_doc, file=None):
  if file:
    if file.endswith(".xml"):
      output_doc.write(file)
  else:
    data = BeautifulSoup(et.tostring(output_doc), "xml")
    print(data.prettify())

def save_xsl(s, file=None):
  if file.endswith(".xsl"):
    if hasattr(s, 'write'):
      xslt_doc = s
    else:
      xslt_doc = et.XML(s).getroottree()
    xslt_doc.write(file)

# Transform:

## II. Make statistics on compounds and hits:

### A. Number of spectra

In [ ]:
xsl_template_spectra_count = '''\
<?xml version="1.0" ?>
<xsl:stylesheet xmlns:xsl="http://www.w3.org/1999/XSL/Transform"
                version="1.0">
<xsl:output method="html"/>

<xsl:template match="/root">
<html>
<head></head>
<body>
    <p>
      Number of spectra: <xsl:value-of select='count(//spectra)'/>
    </p>
</body>
</html>
</xsl:template>


</xsl:stylesheet>
'''

output_doc_spectra_count = transform_xml(xml_file_path, xsl_template_spectra_count)
publish_html(output_doc_spectra_count)

### B. Total number of hits and two columns for a count of correct, incorrect

In [ ]:
xsl_template_hits_count = '''\
<xsl:stylesheet xmlns:xsl="http://www.w3.org/1999/XSL/Transform" version="1.0">
  <xsl:output method="html"/>

  <xsl:template match="/root">
    <html>
      <head></head>
      <body>
        <!-- Total number of hits -->
        <p>Total number of hits: <xsl:value-of select='count(//hits)'/></p>

        <!-- Count of correct hits -->
        <p>Count of correct hits: <xsl:value-of select='count(//hits[cosineMatch_rank=1 or denverMatch_rank=1 or nistMatch_rank=1])'/></p>

        <!-- Count of incorrect hits -->
        <p>Count of incorrect hits: <xsl:value-of select='count(//hits[cosineMatch_rank!=1 and denverMatch_rank!=1 and nistMatch_rank!=1])'/></p>
      </body>
    </html>
  </xsl:template>
</xsl:stylesheet>
'''

output_doc_hits_count = transform_xml(xml_file_path, xsl_template_hits_count)
publish_html(output_doc_hits_count)

### C. An average score of correct and incorrect candidates

In [ ]:
xsl_template_hits_avg = '''\
<xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">

  <xsl:output method="html" indent="yes" />

  <xsl:template match="/">
    <html>
      <head>
        <title>Match Statistics</title>
      </head>
      <body>
        <!-- Initialize counts and sum of scores -->
        <xsl:variable name="correct_hits" select="//hits[cosineMatch_rank=1 or denverMatch_rank=1 or nistMatch_rank=1]" />
        <xsl:variable name="incorrect_hits" select="//hits[cosineMatch_rank!=1 and denverMatch_rank!=1 and nistMatch_rank!=1]" />
        <!-- Calculate average CosineMatch for hits -->
        <xsl:variable name="averageCosineMatch" select="sum($correct_hits/cosineMatch) div count($correct_hits)" />
        <xsl:variable name="averageDenverMatch" select="sum($correct_hits/denverMatch) div count($correct_hits)" />
        <xsl:variable name="averageNistMatch" select="sum($correct_hits/nistMatch) div count($correct_hits)" />


        <xsl:variable name="correctMatches">
          <xsl:value-of select="($averageCosineMatch + $averageDenverMatch + $averageNistMatch) div 3" />
        </xsl:variable>

        <xsl:variable name="averageCosineMatch_inc" select="sum($incorrect_hits/cosineMatch) div count($incorrect_hits)" />
        <xsl:variable name="averageDenverMatch_inc" select="sum($incorrect_hits/denverMatch) div count($incorrect_hits)" />
        <xsl:variable name="averageNistMatch_inc" select="sum($incorrect_hits/nistMatch) div count($incorrect_hits)" />

        <xsl:variable name="incorrectMatches">
          <xsl:value-of select="($averageCosineMatch_inc + $averageDenverMatch_inc + $averageNistMatch_inc) div 3" />
        </xsl:variable>



        <!-- Output results -->
        <p>Average correct Score: <xsl:value-of select="$correctMatches" /></p>
        <p>Average Incorrect Score: <xsl:value-of select="$incorrectMatches" /></p>
      </body>
    </html>
  </xsl:template>


</xsl:stylesheet>
'''

output_doc_hits_avg = transform_xml(xml_file_path, xsl_template_hits_avg)
publish_html(output_doc_hits_avg)

## III. Make statistics on spectra

### A. Total Number of Peaks

In [ ]:
xsl_template_peaks_count = '''\
<?xml version="1.0" ?>
<xsl:stylesheet xmlns:xsl="http://www.w3.org/1999/XSL/Transform" version="1.0">
  <xsl:template match="/">
    <html>
      <head></head>
      <body>
        <p>
          Number of peaks: <xsl:value-of select='count(//peaks)'/>
        </p>
      </body>
    </html>
  </xsl:template>
</xsl:stylesheet>
'''
output_doc_peaks_count = transform_xml(xml_file_path, xsl_template_peaks_count)
publish_html(output_doc_peaks_count)


### B. Number of peaks for three levels of abundance (high, middle, low)

In [ ]:
xsl_template_peaks_abundance = '''\
<?xml version="1.0" ?>
<xsl:stylesheet xmlns:xsl="http://www.w3.org/1999/XSL/Transform" version="1.0">
  <xsl:template match="/">
    <html>
      <head></head>
      <body>
        <p>
          Number of peaks for three levels of abundance (high, middle, low):
        </p>
        <p>
          High: <xsl:value-of select="count(//peaks[abundance > 0.01])"/>
        </p>
        <p>
          Middle: <xsl:value-of select="count(//peaks[abundance &gt; 0.0001 and abundance &lt;= 0.01])"/>
        </p>
        <p>
          Low: <xsl:value-of select="count(//peaks[abundance &lt;= 0.0001])"/>
        </p>
      </body>
    </html>
  </xsl:template>
</xsl:stylesheet>
'''
output_doc_peaks_abundance = transform_xml(xml_file_path, xsl_template_peaks_abundance)
publish_html(output_do_peaks_abundancec)

### C. Number of peaks for bins of m/z such that each bin has a fixed range of masses (lowest and highest bound comes from all peaks in the dataset, inner thresholds for bins are user-defined)

In [ ]:
xsl_template_peaks_bins = '''\
<?xml version="1.0" ?>
<xsl:stylesheet xmlns:xsl="http://www.w3.org/1999/XSL/Transform" version="1.0">
  <xsl:template match="/">
    <html>
      <head></head>
      <body>
        <p>
          Bin 1 (0.1 - 0.25):
          <xsl:value-of select="count(//peaks[mass &gt;= 0.1 and mass &lt; 0.25])"/>
        </p>
        <p>
          Bin 2 (0.25 - 0.4):
          <xsl:value-of select="count(//peaks[mass &gt;= 0.25 and mass &lt; 0.4])"/>
        </p>
        <p>
          Bin 3 (0.4 - 0.55):
          <xsl:value-of select="count(//peaks[mass &gt;= 0.4 and mass &lt; 0.55])"/>
        </p>
        <p>
          Bin 1 (0.55 - 0.7):
          <xsl:value-of select="count(//peaks[mass &gt;= 0.55 and mass &lt; 0.7])"/>
        </p>
        <p>
          Bin 1 (0.7 - 0.85):
          <xsl:value-of select="count(//peaks[mass &gt;= 0.7 and mass &lt; 0.85])"/>
        </p>
        <p>
          Bin 1 (0.85 - 1.0):
          <xsl:value-of select="count(//peaks[mass &gt;= 0.85 and mass &lt; 1.0])"/>
        </p>
      </body>
    </html>
  </xsl:template>
</xsl:stylesheet>
'''
output_doc_peaks_bins = transform_xml(xml_file_path, xsl_template_peaks_bins)
publish_html(output_doc_peaks_bins)

### D. Bold font for m/z bin with the highest number of peaks

In [ ]:
xsl_template_peaks_bins_bold = '''\
<?xml version="1.0" ?>
<xsl:stylesheet xmlns:xsl="http://www.w3.org/1999/XSL/Transform" version="1.0">
  <xsl:template match="/">
    <html>
      <head></head>
      <body>
          <xsl:variable name="bin1" select="count(//peaks[mass &gt;= 0.1 and mass &lt; 0.25])"/>
          <xsl:variable name="bin2" select="count(//peaks[mass &gt;= 0.25 and mass &lt; 0.4])"/>
          <xsl:variable name="bin3" select="count(//peaks[mass &gt;= 0.4 and mass &lt; 0.55])"/>
          <xsl:variable name="bin4" select="count(//peaks[mass &gt;= 0.55 and mass &lt; 0.7])"/>
          <xsl:variable name="bin5" select="count(//peaks[mass &gt;= 0.7 and mass &lt; 0.85])"/>
          <xsl:variable name="bin6" select="count(//peaks[mass &gt;= 0.85 and mass &lt; 1.0])"/>
          <p>
            Bin 1 (0.1 - 0.25):
            <xsl:choose>
              <xsl:when test="$bin1 &gt;= $bin2 and $bin1 &gt;= $bin3">
                <b><xsl:value-of select="$bin1"/></b>
              </xsl:when>
              <xsl:otherwise>
                <xsl:value-of select="$bin1"/>
              </xsl:otherwise>
            </xsl:choose>
          </p>
          <p>
            Bin 2 (0.25 - 0.4):
            <xsl:choose>
              <xsl:when test="$bin2 &gt;= $bin1 and $bin2 &gt;= $bin3">
                <b><xsl:value-of select="$bin2"/></b>
              </xsl:when>
              <xsl:otherwise>
                <xsl:value-of select="$bin2"/>
              </xsl:otherwise>
            </xsl:choose>
          </p>
          <p>
            Bin 3 (0.4 - 0.55):
            <xsl:choose>
              <xsl:when test="$bin3 &gt;= $bin1 and $bin3 &gt;= $bin2">
                <b><xsl:value-of select="$bin3"/></b>
              </xsl:when>
              <xsl:otherwise>
                <xsl:value-of select="$bin3"/>
              </xsl:otherwise>
            </xsl:choose>
          </p>
          <p>
            Bin 4 (0.55 - 0.7):
            <xsl:choose>
              <xsl:when test="$bin3 &gt;= $bin1 and $bin3 &gt;= $bin2">
                <b><xsl:value-of select="$bin3"/></b>
              </xsl:when>
              <xsl:otherwise>
                <xsl:value-of select="$bin3"/>
              </xsl:otherwise>
            </xsl:choose>
          </p>
          <p>
            Bin 5 (0.7 - 0.85):
            <xsl:choose>
              <xsl:when test="$bin3 &gt;= $bin1 and $bin3 &gt;= $bin2">
                <b><xsl:value-of select="$bin3"/></b>
              </xsl:when>
              <xsl:otherwise>
                <xsl:value-of select="$bin3"/>
              </xsl:otherwise>
            </xsl:choose>
          </p>
          <p>
            Bin 6 (0.85 - 1.0):
            <xsl:choose>
              <xsl:when test="$bin3 &gt;= $bin1 and $bin3 &gt;= $bin2">
                <b><xsl:value-of select="$bin3"/></b>
              </xsl:when>
              <xsl:otherwise>
                <xsl:value-of select="$bin3"/>
              </xsl:otherwise>
            </xsl:choose>
          </p>
      </body>
    </html>
  </xsl:template>
</xsl:stylesheet>
'''
output_doc_peaks_bins_bold = transform_xml(xml_file_path, xsl_template_peaks_bins_bold)
publish_html(output_doc_peaks_bins_bold)

### E. *Find the most common m/z location for m/z rounded to whole numbers*

In [ ]:
def create_xml_with_most_common_mz(original_xml_path, new_xml_path):

    tree = ET.parse(original_xml_path)
    root = tree.getroot()

    for spectrum in root.findall(".//spectra"):
        rounded_mass_values = [round(float(mass.text)) for mass in spectrum.findall('.//peaks/mass')]

        if rounded_mass_values:
            most_common_mass = Counter(rounded_mass_values).most_common(1)[0][0]
            most_common_mass_elem = ET.SubElement(spectrum, 'mostCommonRoundedMass')
            most_common_mass_elem.text = str(most_common_mass)

    tree.write(new_xml_path)

original_xml_path = "/content/drive/MyDrive/DSU/synthetic_search_results.xml"
new_xml_path = "/content/drive/MyDrive/DSU/synthetic_search_results_updated.xml"

# Create the new XML file
create_xml_with_most_common_mz(original_xml_path, new_xml_path)
new_xml_path

'/content/drive/MyDrive/DSU/synthetic_search_results_updated.xml'

### F. *Use m/z accuracy to render an updated number of peaks in m/z bins such that peaks bordering the bins will be counted as +0.5 to the bin which he extends through its accuracy.*

In [ ]:
def add_adjusted_bin_counts(original_xml_path, new_xml_path):
    tree = ET.parse(original_xml_path)
    root = tree.getroot()

    for spectrum in root.findall(".//spectra"):
        # Vypočítanie najčastejšieho zaokrúhleného mass
        rounded_mass_values = [round(float(mass.text)) for mass in spectrum.findall('.//peaks/mass')]
        if rounded_mass_values:
            most_common_mass = Counter(rounded_mass_values).most_common(1)[0][0]
            most_common_mass_elem = ET.SubElement(spectrum, 'mostCommonRoundedMass')
            most_common_mass_elem.text = str(most_common_mass)

        # Vypočítanie a pridanie hodnôt bin
        masses = [float(peak.find('mass').text) for peak in spectrum.findall('.//peaks')]
        accuracies = [float(peak.find('accuracy').text) for peak in spectrum.findall('.//peaks')]
        bin_counts = {'bin1': 0, 'bin2': 0, 'bin3': 0, 'bin4': 0, 'bin5': 0, 'bin6': 0}
        bin_ranges = {
            'bin1': (0.1, 0.25),
            'bin2': (0.25, 0.4),
            'bin3': (0.4, 0.55),
            'bin4': (0.55, 0.7),
            'bin5': (0.7, 0.85),
            'bin6': (0.85, 1.0)
        }

        for mass, accuracy in zip(masses, accuracies):
            for bin_name, (lower, upper) in bin_ranges.items():
                # Full inclusion in the bin
                if lower <= mass < upper:
                    bin_counts[bin_name] += 1
                # Partial inclusion due to accuracy
                elif mass - accuracy < upper or  mass + accuracy >= lower :
                    bin_counts[bin_name] += 0.5

        # Pridanie vypočítaných hodnôt bin do elementu spectra
        for bin_name, count in bin_counts.items():
            bin_element = ET.SubElement(spectrum, bin_name)
            bin_element.text = str(count)

    # Uloženie zmeneného XML do nového súboru
    tree.write(new_xml_path)

# Cesty k súborom
original_xml_path = "/content/drive/MyDrive/DSU/synthetic_search_results.xml"
new_xml_path = "/content/drive/MyDrive/DSU/synthetic_search_results_updated_bins.xml"

# Spustenie funkcie
add_adjusted_bin_counts(original_xml_path, new_xml_path)


# Load: Move data from the staging area into the target data storage or publish data for presentation:

## IV. Make a table of compounds where each row represents one compound, and columns will provide statistics retrieved in II.

In [ ]:
xsl_template_compounds = '''\
<xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">

  <xsl:output method="html" indent="yes" />

  <xsl:template match="/">
    <html>
      <head>
        <title>Compounds Table</title>
        <style>
          table {
            border-collapse: collapse;
            width: 100%;
          }
          th, td {
            border: 1px solid #dddddd;
            text-align: left;
            padding: 8px;
          }
          th {
            background-color: #f2f2f2;
          }
        </style>
      </head>
      <body>
        <!-- Select compounds -->
        <xsl:variable name="compounds" select="//recordId" />

        <!-- Output table header -->
        <table>
          <tr>
            <th>Record ID</th>
            <th>Number of spectra</th>
            <th>Number of hits</th>
            <th>count of correct hits</th>
            <th>count of incorrect hits</th>
            <th>Average score of correct hits</th>
            <th>Average score of incorrect hits</th>
          </tr>

          <!-- Iterate over compounds -->
          <xsl:for-each select="$compounds">
            <xsl:variable name="compound" select="ancestor::records | ancestor::recordsrecords" />
            <xsl:variable name="hits" select="$compound//hits" />
            <xsl:variable name="correct_hits" select="$compound//hits[cosineMatch_rank=1 or denverMatch_rank=1 or nistMatch_rank=1]" />
            <xsl:variable name="incorrect_hits" select="$compound//hits[cosineMatch_rank!=1 and denverMatch_rank!=1 and nistMatch_rank!=1]" />

            <!-- Output table row for each compound -->
            <tr>
              <td><xsl:value-of select="." /></td>
              <td><xsl:value-of select="count($compound//spectra)" /></td>
              <td><xsl:value-of select="count($hits)" /></td>

              <td><xsl:value-of select="count($correct_hits)" /></td>
              <td><xsl:value-of select="count($incorrect_hits)" /></td>


              <!-- Calculate average CosineMatch for hits -->
              <xsl:variable name="averageCosineMatch" select="sum($correct_hits/cosineMatch) div count($correct_hits)" />
              <xsl:variable name="averageDenverMatch" select="sum($correct_hits/denverMatch) div count($correct_hits)" />
              <xsl:variable name="averageNistMatch" select="sum($correct_hits/nistMatch) div count($correct_hits)" />


              <xsl:variable name="correctMatches">
                <xsl:value-of select="($averageCosineMatch + $averageDenverMatch + $averageNistMatch) div 3" />
              </xsl:variable>

              <xsl:variable name="averageCosineMatch_inc" select="sum($incorrect_hits/cosineMatch) div count($incorrect_hits)" />
              <xsl:variable name="averageDenverMatch_inc" select="sum($incorrect_hits/denverMatch) div count($incorrect_hits)" />
              <xsl:variable name="averageNistMatch_inc" select="sum($incorrect_hits/nistMatch) div count($incorrect_hits)" />

              <xsl:variable name="incorrectMatches">
                <xsl:value-of select="($averageCosineMatch_inc + $averageDenverMatch_inc + $averageNistMatch_inc) div 3" />
              </xsl:variable>

              <td><xsl:value-of select="$correctMatches" /></td>
              <td><xsl:value-of select="$incorrectMatches" /></td>
            </tr>
          </xsl:for-each>
        </table>
      </body>
    </html>
  </xsl:template>

</xsl:stylesheet>
'''

output_doc_compounds = transform_xml(xml_file_path, xsl_template_compounds)
publish_html(output_doc_compounds)

Record ID,Number of spectra,Number of hits,count of correct hits,count of incorrect hits,Average score of correct hits,Average score of incorrect hits
0,6,16,6,10,0.358752559395802,0.24061255646741
1,19,34,20,14,0.174346034102626,0.171346126627185
10,9,18,9,9,0.421514636026508,0.25945836530728
100,30,60,38,22,0.302326164157339,0.265707773941709
101,12,96,19,77,0.387854895421115,0.220725347581453
103,8,14,8,6,0.152281405586416,0.107022618926435
104,18,38,24,14,0.196883352022125,0.198091593026646
108,3,6,3,3,0.300437917494704,0.130343801329151
11,21,24,21,3,0.223652289057493,0.360111124212359
111,12,24,13,11,0.675834024921515,0.336795419919242


## V. Make a table of spectra where each row represents one spectrum, and columns will contain statistics retrieved in III.

In [ ]:
xsl_template_spectra = '''\
<xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">
  <xsl:output method="html" indent="yes" />

  <xsl:template match="/">
    <html>
      <head>
        <title>Spectra Table</title>
        <style>
          table {
            border-collapse: collapse;
            width: 100%;
          }
          th, td {
            border: 1px solid #dddddd;
            text-align: left;
            padding: 8px;
          }
          th {
            background-color: #f2f2f2;
          }
        </style>
      </head>
      <body>
        <xsl:for-each select="//records/recordId | //recordsrecords/recordId">
          <xsl:variable name="currentRecordId" select="."/>
          <h2>Record ID: <xsl:value-of select="$currentRecordId"/></h2>
          <table>
          <tr>
            <th>Spectrum ID</th>
            <th>Total Peaks</th>
            <th>High Abundance</th>
            <th>Middle Abundance</th>
            <th>Low Abundance</th>
            <th>Bin 1</th>
            <th>Bin 2</th>
            <th>Bin 3</th>
            <th>Bin 4</th>
            <th>Bin 5</th>
            <th>Bin 6</th>
            <th>Most Common m/z</th>
          </tr>
          <xsl:for-each select="//records[recordId = $currentRecordId]/spectrumCollections/spectra/spectrumId |
              //recordsrecords[recordId = $currentRecordId]/spectrumCollections/spectra/spectrumId">
            <xsl:variable name="bin1" select="count(../peaks[mass &gt;= 0.1 and mass &lt; 0.25])"/>
            <xsl:variable name="bin2" select="count(../peaks[mass &gt;= 0.25 and mass &lt; 0.4])"/>
            <xsl:variable name="bin3" select="count(../peaks[mass &gt;= 0.4 and mass &lt; 0.55])"/>
            <xsl:variable name="bin4" select="count(../peaks[mass &gt;= 0.55 and mass &lt; 0.7])"/>
            <xsl:variable name="bin5" select="count(../peaks[mass &gt;= 0.7 and mass &lt; 0.85])"/>
            <xsl:variable name="bin6" select="count(../peaks[mass &gt;= 0.85 and mass &lt; 1.0])"/>
            <tr>
              <td><xsl:value-of select="."/></td>
              <td><xsl:value-of select="count(../peaks)"/></td>
              <td><xsl:value-of select="count(../peaks[abundance > 0.001])"/></td>
              <td><xsl:value-of select="count(../peaks[abundance &gt; 0.00001 and abundance &lt;= 0.001])"/></td>
              <td><xsl:value-of select="count(../peaks[abundance &lt;= 0.00001])"/></td>
              <td>
                <xsl:choose>
                  <xsl:when test="$bin1 &gt;= $bin2 and $bin1 &gt;= $bin3 and $bin1 &gt;= $bin4 and $bin1 &gt;= $bin5 and $bin1 &gt;= $bin6">
                    <b><xsl:value-of select="$bin1"/></b>
                  </xsl:when>
                  <xsl:otherwise>
                    <xsl:value-of select="$bin1"/>
                  </xsl:otherwise>
                </xsl:choose>
              </td>
              <td>
                <xsl:choose>
                  <xsl:when test="$bin2 &gt;= $bin1 and $bin2 &gt;= $bin3 and $bin2 &gt;= $bin4 and $bin2 &gt;= $bin5 and $bin2 &gt;= $bin6">
                    <b><xsl:value-of select="$bin2"/></b>
                  </xsl:when>
                  <xsl:otherwise>
                    <xsl:value-of select="$bin2"/>
                  </xsl:otherwise>
                </xsl:choose>
              </td>
              <td>
                <xsl:choose>
                  <xsl:when test="$bin3 &gt;= $bin1 and $bin3 &gt;= $bin2 and $bin3 &gt;= $bin4 and $bin3 &gt;= $bin5 and $bin3 &gt;= $bin6">
                    <b><xsl:value-of select="$bin3"/></b>
                  </xsl:when>
                  <xsl:otherwise>
                    <xsl:value-of select="$bin3"/>
                  </xsl:otherwise>
                </xsl:choose>
              </td>
              <td>
                <xsl:choose>
                  <xsl:when test="$bin4 &gt;= $bin1 and $bin4 &gt;= $bin2 and $bin4 &gt;= $bin3 and $bin4 &gt;= $bin5 and $bin4 &gt;= $bin6">
                    <b><xsl:value-of select="$bin4"/></b>
                  </xsl:when>
                  <xsl:otherwise>
                    <xsl:value-of select="$bin4"/>
                  </xsl:otherwise>
                </xsl:choose>
              </td>
              <td>
                <xsl:choose>
                  <xsl:when test="$bin5 &gt;= $bin1 and $bin5 &gt;= $bin2 and $bin5 &gt;= $bin3 and $bin5 &gt;= $bin4 and $bin5 &gt;= $bin6">
                    <b><xsl:value-of select="$bin5"/></b>
                  </xsl:when>
                  <xsl:otherwise>
                    <xsl:value-of select="$bin5"/>
                  </xsl:otherwise>
                </xsl:choose>
              </td>
              <td>
                <xsl:choose>
                  <xsl:when test="$bin6 &gt;= $bin1 and $bin6 &gt;= $bin2 and $bin6 &gt;= $bin3 and $bin6 &gt;= $bin4 and $bin6 &gt;= $bin5">
                    <b><xsl:value-of select="$bin6"/></b>
                  </xsl:when>
                  <xsl:otherwise>
                    <xsl:value-of select="$bin6"/>
                  </xsl:otherwise>
                </xsl:choose>
              </td>
              <td><xsl:value-of select="../mostCommonRoundedMass"/></td>
            </tr>
          </xsl:for-each>
        </table>
        </xsl:for-each>
      </body>
    </html>
  </xsl:template>
</xsl:stylesheet>
'''

output_doc_spectra = transform_xml(xml_file_path, xsl_template_spectra)
publish_html(output_doc_spectra)

Spectrum ID,Total Peaks,High Abundance,Middle Abundance,Low Abundance,Bin 1,Bin 2,Bin 3,Bin 4,Bin 5,Bin 6,Most Common m/z
1295,12,6,3,3,3,3,0,3,0,3,0
155,9,1,2,6,1,2,2,2,2,0,0
1623,12,6,3,3,3,3,0,3,0,3,0
364,39,6,9,24,6,6,6,6,9,6,1
457,45,6,18,21,15,6,6,12,3,3,0
884,45,3,15,27,6,6,9,12,9,3,1
Spectrum ID,Total Peaks,High Abundance,Middle Abundance,Low Abundance,Bin 1,Bin 2,Bin 3,Bin 4,Bin 5,Bin 6,Most Common m/z
1059,232,24,76,132,52,96,32,36,16,0,0
1184,154,12,68,74,102,50,2,0,0,0,0
1332,146,8,82,56,106,40,0,0,0,0,0


### Upload bins based on accuracy

In [ ]:
xsl_template_compounds = '''\
<xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">
  <xsl:output method="html" indent="yes" />

  <xsl:template match="/">
    <html>
      <head>
        <title>Spectra Table</title>
        <style>
          table {
            border-collapse: collapse;
            width: 100%;
          }
          th, td {
            border: 1px solid #dddddd;
            text-align: left;
            padding: 8px;
          }
          th {
            background-color: #f2f2f2;
          }
        </style>
      </head>
      <body>
        <xsl:for-each select="//records/recordId | //recordsrecords/recordId">
          <xsl:variable name="currentRecordId" select="."/>
          <h2>Record ID: <xsl:value-of select="$currentRecordId"/></h2>
          <table>
          <tr>
            <th>Spectrum ID</th>
            <th>Total Peaks</th>
            <th>High Abundance</th>
            <th>Middle Abundance</th>
            <th>Low Abundance</th>
            <th>Bin 1</th>
            <th>Bin 2</th>
            <th>Bin 3</th>
            <th>Bin 4</th>
            <th>Bin 5</th>
            <th>Bin 6</th>
            <th>Most Common m/z</th>
          </tr>
          <xsl:for-each select="//records[recordId = $currentRecordId]/spectrumCollections/spectra/spectrumId |
              //recordsrecords[recordId = $currentRecordId]/spectrumCollections/spectra/spectrumId">
            <xsl:variable name="bin1" select="../bin1"/>
            <xsl:variable name="bin2" select="../bin2"/>
            <xsl:variable name="bin3" select="../bin3"/>
            <xsl:variable name="bin4" select="../bin4"/>
            <xsl:variable name="bin5" select="../bin5"/>
            <xsl:variable name="bin6" select="../bin6"/>
            <tr>
              <td><xsl:value-of select="."/></td>
              <td><xsl:value-of select="count(../peaks)"/></td>
              <td><xsl:value-of select="count(../peaks[abundance > 0.001])"/></td>
              <td><xsl:value-of select="count(../peaks[abundance &gt; 0.00001 and abundance &lt;= 0.001])"/></td>
              <td><xsl:value-of select="count(../peaks[abundance &lt;= 0.00001])"/></td>
              <td>
                <xsl:choose>
                  <xsl:when test="$bin1 &gt;= $bin2 and $bin1 &gt;= $bin3 and $bin1 &gt;= $bin4 and $bin1 &gt;= $bin5 and $bin1 &gt;= $bin6">
                    <b><xsl:value-of select="$bin1"/></b>
                  </xsl:when>
                  <xsl:otherwise>
                    <xsl:value-of select="$bin1"/>
                  </xsl:otherwise>
                </xsl:choose>
              </td>
              <td>
                <xsl:choose>
                  <xsl:when test="$bin2 &gt;= $bin1 and $bin2 &gt;= $bin3 and $bin2 &gt;= $bin4 and $bin2 &gt;= $bin5 and $bin2 &gt;= $bin6">
                    <b><xsl:value-of select="$bin2"/></b>
                  </xsl:when>
                  <xsl:otherwise>
                    <xsl:value-of select="$bin2"/>
                  </xsl:otherwise>
                </xsl:choose>
              </td>
              <td>
                <xsl:choose>
                  <xsl:when test="$bin3 &gt;= $bin1 and $bin3 &gt;= $bin2 and $bin3 &gt;= $bin4 and $bin3 &gt;= $bin5 and $bin3 &gt;= $bin6">
                    <b><xsl:value-of select="$bin3"/></b>
                  </xsl:when>
                  <xsl:otherwise>
                    <xsl:value-of select="$bin3"/>
                  </xsl:otherwise>
                </xsl:choose>
              </td>
              <td>
                <xsl:choose>
                  <xsl:when test="$bin4 &gt;= $bin1 and $bin4 &gt;= $bin2 and $bin4 &gt;= $bin3 and $bin4 &gt;= $bin5 and $bin4 &gt;= $bin6">
                    <b><xsl:value-of select="$bin4"/></b>
                  </xsl:when>
                  <xsl:otherwise>
                    <xsl:value-of select="$bin4"/>
                  </xsl:otherwise>
                </xsl:choose>
              </td>
              <td>
                <xsl:choose>
                  <xsl:when test="$bin5 &gt;= $bin1 and $bin5 &gt;= $bin2 and $bin5 &gt;= $bin3 and $bin5 &gt;= $bin4 and $bin5 &gt;= $bin6">
                    <b><xsl:value-of select="$bin5"/></b>
                  </xsl:when>
                  <xsl:otherwise>
                    <xsl:value-of select="$bin5"/>
                  </xsl:otherwise>
                </xsl:choose>
              </td>
              <td>
                <xsl:choose>
                  <xsl:when test="$bin6 &gt;= $bin1 and $bin6 &gt;= $bin2 and $bin6 &gt;= $bin3 and $bin6 &gt;= $bin4 and $bin6 &gt;= $bin5">
                    <b><xsl:value-of select="$bin6"/></b>
                  </xsl:when>
                  <xsl:otherwise>
                    <xsl:value-of select="$bin6"/>
                  </xsl:otherwise>
                </xsl:choose>
              </td>
              <td><xsl:value-of select="../mostCommonRoundedMass"/></td>
            </tr>
          </xsl:for-each>
        </table>
        </xsl:for-each>
      </body>
    </html>
  </xsl:template>
</xsl:stylesheet>
'''

output_doc_compounds = transform_xml(xml_file_path, xsl_template_compounds)
publish_html(output_doc_compounds)

Spectrum ID,Total Peaks,High Abundance,Middle Abundance,Low Abundance,Bin 1,Bin 2,Bin 3,Bin 4,Bin 5,Bin 6,Most Common m/z
1295,12,6,3,3,7.5,7.5,6.0,7.5,6.0,7.5,0
155,9,1,2,6,5.0,5.5,5.5,5.5,5.5,4.5,0
1623,12,6,3,3,7.5,7.5,6.0,7.5,6.0,7.5,0
364,39,6,9,24,22.5,22.5,22.5,22.5,24.0,22.5,1
457,45,6,18,21,30.0,25.5,25.5,28.5,24.0,24.0,0
884,45,3,15,27,25.5,25.5,27.0,28.5,27.0,24.0,1
Spectrum ID,Total Peaks,High Abundance,Middle Abundance,Low Abundance,Bin 1,Bin 2,Bin 3,Bin 4,Bin 5,Bin 6,Most Common m/z
1059,232,24,76,132,142.0,164.0,132.0,134.0,124.0,116.0,0
1184,154,12,68,74,128.0,102.0,78.0,77.0,77.0,77.0,0
1332,146,8,82,56,126.0,93.0,73.0,73.0,73.0,73.0,0


## VI. *Shed some light on the relationship between normalized collision energy and peaks distribution*

In [ ]:

# Parse the XML file
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Extract NCE values and corresponding number of peaks
nce_values = [float(spectrum.find(".//metadata/nce").text) for spectrum in root.findall(".//spectra")]
peak_counts = [len(spectrum.findall(".//peaks")) for spectrum in root.findall(".//spectra")]

# Plotting the relationship between NCE and number of peaks
plt.figure(figsize=(10, 6))
plt.scatter(nce_values, peak_counts, color='blue', alpha=0.5)
plt.title('Relationship between Normalized Collision Energy (NCE) and Number of Peaks')
plt.xlabel('Normalized Collision Energy (NCE)')
plt.ylabel('Number of Peaks')
plt.grid(True)
plt.show()


> The scatter plot shows the relationship between the Normalized Collision Energy (NCE) and the number of peaks for each spectrum in the dataset. Each point on the plot represents a spectrum, and its position is determined by both the NCE value and the corresponding number of peaks in that spectrum. This plot helps visualize how the NCE is distributed concerning the number of peaks in the spectra.


## VII. *Discuss important patterns in data.*

**Spectrum Information**

> Each spectrum has associated metadata, including analyzer type, MS stage, polarity, scan range, NCE (normalized collision energy), ion activation, isolation center, and isolation width.

**Hits Information**

> Hits are associated with each spectrum, providing details about matches with other records.

**Peaks Information**

> Peaks within each spectrum are characterized by mass, abundance, and accuracy.

> Mass and abundance values vary for different peaks within a spectrum.

**Repetition in Data**

> Some information, such as spectrum and hits details, is repeated in the XML structure, possibly indicating multiple hits or analyses for the same spectrum.

**Match Rankings**

> The ranking information (cosineMatch_rank) suggests the order or significance of matches for a given spectrum

**Abundance Distribution**

> Abundance values for peaks vary, ranging from very small values (e-16) to larger ones, suggesting a diverse set of compounds or ions.

**Match Scores**

> The match scores (cosine, Denver, NIST) provide insights into the similarity or matching degree between different spectra.

**Mass Accuracy**

> Mass accuracy values for precursor peaks and individual peaks reflect the precision of the mass measurements.

**Isolation Parameters**

> Isolation center and isolation width provide information about the selection of precursor ions during the analysis.

